In [ ]:
# @title
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [ ]:
# @title
#Use this and above  code cell if you're using Kaggle notebook
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [18]:
import os
# Get the number of CPU cores
cpu_cores = os.cpu_count()

print("Number of CPU cores:", cpu_cores)


Number of CPU cores: 2


In [1]:
#Use the below cells if you're using colab and import data from Kaggle website using Kaggle api

from google.colab import userdata

os.environ['KAGGLE_USERNAME'] = userdata.get('username')
os.environ['KAGGLE_KEY'] = userdata.get('key')


In [2]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 19% 5.00M/25.7M [00:00<00:00, 36.0MB/s]
100% 25.7M/25.7M [00:00<00:00, 124MB/s] 


In [3]:
!unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [5]:
import pandas as pd
df = pd.read_csv('/content/IMDB Dataset.csv')
print(df.shape)
df.head()


(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
column = 'review'
df_obj = pd.DataFrame(df[column])
df_obj['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df_obj.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [7]:
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder

# Download nltk stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
# Preprocess the text

def preprocess_text(text):
    words = [word.lower() for word in text.split() if word.isalpha() and word.lower() not in stop_words]
    return ' '.join(words)

#Apply preprocessing to the data
df_obj['review'] = df_obj['review'].apply(preprocess_text)

In [9]:
df_obj.head()

,review,sentiment
0,one reviewers mentioned watching oz episode ex...,1
1,wonderful little filming technique fashion giv...,1
2,thought wonderful way spend time hot summer si...,1
3,basically family little boy thinks zombie clos...,0
4,petter time visually stunning film mattei offe...,1


from matplotlib import pyplot as plt
_df_0['sentiment'].plot(kind='hist', bins=20, title='sentiment')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('review').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['sentiment'].plot(kind='line', figsize=(8, 4), title='sentiment')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_3['review'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_3, x='sentiment', y='review', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [10]:
df_obj.shape

(50000, 2)

In [11]:
from sklearn.model_selection import train_test_split

# Split the data set for training
X_train, X_test, y_train, y_test = train_test_split(df_obj['review'], df['sentiment'], test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)

(40000,)
(10000,)


In [12]:
#Create a Pipeline with TF-IDF vectorizer and a classifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('tfidf',TfidfVectorizer(max_features=5000)),
    ('classifier',RandomForestClassifier(n_estimators=100, random_state=42))
])

In [13]:
#Train the model using training set

pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=5000)),
                ('classifier', RandomForestClassifier(random_state=42))])

In [14]:
# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report

y_pred = pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

classification_rep = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_rep)

Accuracy: 0.83
Classification Report:
               precision    recall  f1-score   support

    negative       0.82      0.84      0.83      4961
    positive       0.84      0.83      0.83      5039

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



Find the best Model by Cross validation

In [15]:
pipeline2 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', RandomForestClassifier())
])

In [16]:
from sklearn.model_selection import GridSearchCV

#Define the Parameter Grid for tuning
param_grid = {
    'tfidf__max_features':[5000,10000, 20000],
    'classifier__n_estimators':[50,100, 200],
    'classifier__max_depth':[5,10,15],
}

In [17]:
#Perform grid-search cross-validation

grid_search = GridSearchCV(pipeline2, param_grid, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('classifier',
                                        RandomForestClassifier())]),
             param_grid={'classifier__max_depth': [5, 10, 15],
                         'classifier__n_estimators': [50, 100, 200],
                         'tfidf__max_features': [5000, 10000, 20000]})

In [ ]:
#Calculate each model accuracy rate
for i in range(len(grid_search.cv_results_['mean_test_score'])):
    print(f"Model {i+1} accuracy:{grid_search.cv_results_['mean_test_score'][i]:.4f}")


In [ ]:
# get the best model
best_model = grid_search.best_estimator_

In [ ]:
import joblib
#Optional: Save the best model
filename = 'random_forest_model.joblib'
joblib.dump(best_model, filename)

['random_forest_model.joblib']

In [ ]:
#Evaluate the model
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

classification_rep = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_rep)

Accuracy: 0.83
Classification Report:
               precision    recall  f1-score   support

    negative       0.84      0.80      0.82      4961
    positive       0.81      0.85      0.83      5039

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



### Pipeline2 with Logistics regression

In [ ]:
from sklearn.linear_model import LogisticRegression

#Pipeline with TF-IDF Vectorizer and Logistic Regression
pipeline3=Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('lr', LogisticRegression())
])

#Define the parameter grid
param_grid2 = {
    'tfidf__max_features':[5000, 10000, 20000],
    'lr__C':[0.1, 1.0, 10],
    'lr__penalty':['l1','l2'],
    'lr__solver':['liblinear', 'libgfs']
}


In [ ]:
#Perform Grid-Search Cross Validation

grid_search2 = GridSearchCV(pipeline3, param_grid2, cv=5)
grid_search2.fit(X_train, y_train)

In [ ]:
#Calculate each model accuracy rate

for i in range(len(grid_search2.cv_results_['mean_test_score'])):
    print(f"Model {i+1} accuracy:{grid_search2.cv_results_['mean_test_score'][i]:.4f}")

In [ ]:
# get the best model
best_model2 = grid_search2.best_estimator_

In [ ]:
#Evaluate the model
y_pred = best_model2.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

classification_rep = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_rep)

In [ ]:
#Optional:Save the best Logistic Regression Classifier model
filename = 'logistic_regression_model.joblib'
joblib.dump(best_model2, filename)

#### Tuning the LogisticRegression Model with best hyper parameter and comapre with Grid Search CV

In [ ]:
pipeline4 = Pipeline([
    ('tfidf',TfidfVectorizer(max_features=10000)),
    ('lr', LogisticRegression(C=10, penalty=12, solver='liblinear'))
])

In [ ]:
pipeline4 = pipeline4.fit(X_train, y_train)

In [ ]:
#Evaluate the model
y_pred = best_model2.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

classification_rep = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_rep)